In [3]:
!pip install emoji
!pip install tensorflow
!pip install transformers

     |████████████████████████████████| 175 kB 15.5 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=b28da56fc0ba3b0d4680da20a61200f69195ac08ecbb81bafccd5ba16cfee0a4
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |████████████████████████████████| 462 kB 15.9 MB/s 
     |████████████████████████████████| 4.0 MB 14.6 MB/s 
     |████████████████████████████████| 77 kB 9.1 MB/s 
     |████████████████████████████████| 596 kB 80.3 MB/s 
     |████████████████████████████████| 6.6 MB 86.5 MB/s 
     |████████████████████████████████| 895 kB 72.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

model = TFAutoModelForSequenceClassification.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at rabindralamsal/finetuned-bertweet-sentiment-analysis.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [6]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:
link = 'https://drive.google.com/file/d/17-X8-Fg4B11bqxt_fZ5B-X2WzeuRky0f/view?usp=sharing' 

In [8]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Train_Feb.csv') 
 
df = pd.read_csv('Train_Feb.csv')
print(df)

                                                      text
0        @k_c12345 Brother this called link 👇\nhttps://...
1        Ukraine invasion: Who is President Volodymyr Z...
2        So it now looks like we're in a new phase of t...
3        @vanderbellen @karlnehammer TOP RUSSIAN AGGRES...
4        @ZelenskyyUa @EmmanuelMacron Ukraine won't fal...
...                                                    ...
1782546  @UNESCO Dear UNESCO. I'm a peaceful citizen of...
1782547  This is not Ukraine vs Russia war. This is a c...
1782548  Russia 🇷🇺 is warn... Don't provoke the 4nation...
1782549  Africa Report: How Africa factors in #RussiaUk...
1782550  @BanteSaket @LukeMackay13 @Reporter_indian I l...

[1782551 rows x 1 columns]


In [9]:
df['text'] = df['text'].astype('str') 

In [10]:
#df_sample = df.sample(n=50000, random_state=2)

In [11]:
#df_sample

In [12]:
docs = list(df.loc[:, "text"].values)

In [13]:
#get index for halving dataset at index: 178255
from math import floor
floor(len(docs) / 10)


178255

In [14]:
178255 * 9

1604295

In [15]:
docs = docs[1426040:1604295]

In [16]:
from tqdm import tqdm
scores = []

for i in tqdm(range(len(docs))):
    input = tokenizer.encode(docs[i], return_tensors="tf")
    output = model.predict(input)[0]
    prediction = tf.nn.softmax(output, axis=1).numpy()
    sentiment = np.argmax(prediction)
    
    scores.append({"Label": sentiment,
                       "Negative": prediction[0,0],
                       "Neutral": prediction[0,1],
                       "Positive": prediction[0,2]
                  })

100%|██████████| 178255/178255 [2:45:11<00:00, 17.99it/s]


In [17]:
df = df[1426040:1604295]

In [18]:
sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score.set_index(df.index))

In [19]:
df

,text,Label,Negative,Neutral,Positive
1426040,#SupportUkraine @NastiaPav (Russian Tennis 🎾 P...,2,0.001987,0.014484,0.983528
1426041,@I_Arquam 🤔🤔I apologize but I really don't und...,0,0.971006,0.025687,0.003307
1426042,@TomCottonAR You can’t be this ignorant to thi...,0,0.973429,0.022952,0.003620
1426043,@DmytroKuleba Ukraine was an ally of Hitler an...,1,0.029237,0.800634,0.170130
1426044,More and more sports are following the appeal ...,1,0.421203,0.572128,0.006669
...,...,...,...,...,...
1604290,#TheWarDebate | 'The NATO &amp; US seem set to...,0,0.531574,0.463402,0.005024
1604291,"@LatestAnonPress US, Israel, NATO Bombing othe...",0,0.966447,0.030648,0.002905
1604292,@riabronsgeest @KremlinRussia_E People also li...,0,0.819728,0.176939,0.003333
1604293,"@ianbremmer So that NATO can box in russia, if...",0,0.949124,0.048219,0.002657


In [20]:
from google.colab import drive
drive.mount('drive')

df.to_csv('finetuned_Bertweet_SA_Feb_Train_p9.csv')
!cp finetuned_Bertweet_SA_Feb_Train_p9.csv "drive/My Drive/Colab Notebooks"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
